In [105]:
import json
import random

from nltk.tokenize import regexp_tokenize
from edits_generator import get_alphabet, generate_k_random_candidates_efficient

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [106]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Embedding of words based on edit similarity

Loading training words for embeddings

In [107]:
with open('data/articles_cleaned_vocabulary.json', 'r', encoding="utf-8") as f:
    vocabulary_set = set([d[0] for d in json.load(f)])

pattern = r'([0-9]+|[а-я]+)'
with open('data/bgjargon_words.json', 'r', encoding="utf-8") as f:
    vocabulary_set.union(set([token for word in json.load(f) for token in regexp_tokenize(word, pattern)]))

vocabulary = list(vocabulary_set)
vocabulary = random.choices(vocabulary, k=5000)

Loading training comments and extracting edit alphabet

In [108]:
with open('data/blitz_comments_classified.json', 'r', encoding="utf-8") as f:
    gold_comments = [comment_record[0] for comment_record in json.load(f) if len(comment_record) > 2]

SOW_token = ''
EOW_token = ''

alphabet = get_alphabet(gold_comments)
alphabet += SOW_token
alphabet += EOW_token

char2ind = {}
for i, word in enumerate(alphabet):
    char2ind[word] = i

In [109]:
n_alphabet = len(alphabet) + 1
n_output = len(vocabulary)
MAX_LENGTH = 20

In [110]:
generate_k_random_candidates_efficient('ябълка', 10, alphabet)

['ябплк',
 'яъблкaа',
 'яълkа',
 'Къблка',
 'яъкла',
 'ябэка',
 'ябъл%U',
 'яб\x03ка',
 'ябълчкl',
 'ьбъла']

## Encoder Decoder RNN
Trained to de-noise the candidates

In [111]:
pairs = [(candidate, word)
    for word in vocabulary
    for candidate in generate_k_random_candidates_efficient(word, 10, alphabet)
    if len(candidate) < MAX_LENGTH and len(word) < MAX_LENGTH
]

In [112]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

    def save(self, filepath):
        torch.save(self.state_dict(), filepath)

    @classmethod
    def load(cls, filepath, input_size, hidden_size, dropout_p=0.1):
        model = cls(input_size, hidden_size, dropout_p)
        model.load_state_dict(torch.load(filepath))
        return model

In [113]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(char2ind[SOW_token])
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

## Training

In [114]:
def indexesFromWord(word):
    return [char2ind[c] for c in word]

def tensorFromWord(word):
    indexes = indexesFromWord(word)
    indexes.append(char2ind[EOW_token])
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromWord(pair[0])
    target_tensor = tensorFromWord(pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromWord(inp)
        tgt_ids = indexesFromWord(tgt)
        inp_ids.append(char2ind[EOW_token])
        tgt_ids.append(char2ind[EOW_token])
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return train_dataloader

In [115]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [116]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100):

    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('(%d %d%%) %.4f' % (epoch, epoch / n_epochs * 100, print_loss_avg))

In [118]:
hidden_size = 128
batch_size = 128

train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(n_alphabet, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, n_alphabet).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5)

(5 6%) 0.6598
(10 12%) 0.3032
(15 18%) 0.2379
(20 25%) 0.2015
(25 31%) 0.1769
(30 37%) 0.1587
(35 43%) 0.1451
(40 50%) 0.1338
(45 56%) 0.1252
(50 62%) 0.1179
(55 68%) 0.1116
(60 75%) 0.1065
(65 81%) 0.1019
(70 87%) 0.0974
(75 93%) 0.0945
(80 100%) 0.0912


In [119]:
encoder.eval()
decoder.eval()

def evaluate(word):
    with torch.no_grad():
        input_tensor = tensorFromWord(word)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_chars = []
        for idx in decoded_ids:
            c = alphabet[idx.item()]
            if c == EOW_token:
                break
            decoded_chars.append(c)
    return "".join(decoded_chars)

Saving model

In [148]:
encoder.save("data/embedding_encoder_5000.pth")

Loading model

In [160]:
encoder = EncoderRNN.load("data/embedding_encoder_5000.pth", n_alphabet, hidden_size)
encoder.to(device)
encoder.eval()

EncoderRNN(
  (embedding): Embedding(167, 128)
  (gru): GRU(128, 128, batch_first=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

Getting the produced embedding from the RNN

In [150]:
def embedding(word):
    with torch.no_grad():
        input_tensor = tensorFromWord(word)
        _, encoder_hidden = encoder(input_tensor)
    return encoder_hidden

In [151]:
def euclidean_distance(tensor1, tensor2):
    return (tensor1.flatten() - tensor2.flatten()).pow(2).sum().sqrt()

def cosine_similarity(tensor1, tensor2):
    cos = nn.CosineSimilarity(dim=0)
    return cos(tensor1.flatten(), tensor2.flatten())

Comparing similarity of embeddings of similar words

In [161]:
print(euclidean_distance(embedding('аааббсс'), embedding('аабс')))
print(cosine_similarity(embedding('аааббсс'), embedding('аабс')))
print(euclidean_distance(embedding('аааббсс'), embedding('корса')))
print(cosine_similarity(embedding('аааббсс'), embedding('корса')))

tensor(2.8913, device='cuda:0')
tensor(0.9237, device='cuda:0')
tensor(7.1998, device='cuda:0')
tensor(0.5232, device='cuda:0')


In [162]:
euclidean_distance(embedding('тарикат'), embedding('абонат'))

tensor(6.8839, device='cuda:0')

In [163]:
cosine_similarity(embedding('таноба'), embedding('абонат'))

tensor(0.6317, device='cuda:0')